We have some ideas about the exact matching problem but in reality we need to got for approximate algorithms. The Boyer-Moore algorithm is highly efficient for exact string matching because it often skips large sections of the text when searching for a pattern. In Genomic Data Science, where you're frequently scanning massive DNA sequences for specific patterns or motifs, Boyer-Moore's ability to reduce the number of comparisons can significantly speed up the search process compared to more naive methods. 

# Advantages of the Boyer-Moore Algorithm in Genomic Data Science

The Boyer-Moore algorithm offers several advantages that make it particularly well-suited for pattern matching in genomic datasets:

- **Efficiency:**  
  It often operates in sublinear time on average by skipping large portions of the sequence during the search, which is ideal for scanning large genomic datasets.

- **Reduced Comparisons:**  
  Leveraging heuristics such as the bad-character and good-suffix rules, it minimizes the number of character comparisons, thereby speeding up the matching process.

- **Effective Preprocessing:**  
  The algorithm preprocesses the pattern once in linear time, which is especially beneficial when the same pattern is searched multiple times across vast genomic data.

- **Scalability:**  
  Its computational efficiency makes it highly scalable, suitable for handling massive DNA sequences where performance is critical.

- **Adaptability:**  
  The core principles of Boyer-Moore can be extended to approximate matching, addressing the needs of biological sequence analysis where perfect matches are not always expected.

Overall, these advantages make the Boyer-Moore algorithm a powerful tool for efficient and effective pattern matching in genomic research.


# Implement Boyer Moore

Gusfield, Dan. Algorithms on strings, trees and sequences: computer science and computational biology. Cambridge university press, 1997.

In [2]:
"""
This module implements advanced string matching and preprocessing algorithms based on Gusfield's
techniques. These algorithms are especially useful in applications such as genomic data analysis,
where efficient pattern matching is crucial.

The module includes implementations of:

- **Z-Algorithm (`z_array`)**:  
  Computes the Z-array for a given string `s`. The Z-array is defined such that `Z[k]` is the length
  of the longest substring starting at position `k` that matches a prefix of `s`. This is based on 
  Gusfield's Theorem 1.4.1.

- **N-Array (`n_array`)**:  
  Computes the N-array for a string `s` by reversing `s`, computing its Z-array, and then reversing
  the result. The N-array is useful for constructing the good suffix tables.

- **Big L' Array (`big_l_prime_array`)**:  
  Constructs the L' array for a pattern `p` using its N-array `n`. For each index `i`, 
  `L'[i]` is defined as the largest index `j` less than `len(p)` such that `N[j]` equals the length 
  of the suffix of `p` starting at `i` (Gusfield Theorem 2.2.2).

- **Big L Array (`big_l_array`)**:  
  Constructs the L array for a pattern `p` using the previously computed L' array. For each index `i`, 
  `L[i]` is the largest index `j` less than `len(p)` such that `N[j]` is greater than or equal to the 
  length of the suffix of `p` starting at `i`.

- **Small l' Array (`small_l_prime_array`)**:  
  Constructs the small l' array from the N-array. Each entry in this array represents the length 
  of the longest substring starting at that position that is also a prefix of the pattern (Gusfield 
  Theorem 2.2.4).

- **Good Suffix Table (`good_suffix_table`)**:  
  Combines the results from the N-array, Big L' array, Big L array, and small l' array to produce 
  the necessary tables for the good suffix rule in pattern matching.

- **Good Suffix Shift Functions**:
    - `good_suffix_mismatch(i, big_l_prime, small_l_prime)`:  
      Given a mismatch at offset `i` in the pattern, this function calculates the shift distance 
      based on the good suffix rule using the Big L' and small l' arrays.
    - `good_suffix_match(small_l_prime)`:  
      When the pattern fully matches the text, this function returns the shift amount as determined 
      by the good suffix rule using the small l' array.

- **Dense Bad Character Table (`dense_bad_char_tab`)**:  
  Builds a dense table for the bad character rule. Given a pattern and an ordered alphabet mapping, 
  the table is indexed by the pattern offset and character, facilitating quick lookup during mismatches.

- **BoyerMoore Class**:  
  Encapsulates the Boyer-Moore string matching algorithm. Upon initialization, it precomputes:
    - A mapping from alphabet characters to integer indices.
    - The dense bad character table.
    - The good suffix tables (Big L and small l' arrays).
  
  The class provides methods for:
    - `bad_character_rule(i, c)`:  
      Determines the shift distance based on the bad character rule when a mismatch occurs at index `i`
      with character `c`.
    - `good_suffix_rule(i)`:  
      Determines the shift distance based on the good suffix rule when a mismatch occurs at index `i`.
    - `match_skip()`:  
      Returns the shift distance to use when a full match of the pattern is found.

**Reference:**
Gusfield, D. (1997). *Algorithms on Strings, Trees, and Sequences: Computer Science and Computational Biology*.
Cambridge University Press.
"""

import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z


def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp


def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

#### Checking for the correctness of the functions

In [9]:
import time
start = time.perf_counter()
# GCTAGCTCTACGAGTCTA
p = 'TCAA'
p_bm = BoyerMoore(p, alphabet='ACGT')
print(p_bm.bad_character_rule(2, 'T'))

end = time.perf_counter()
print(f"Time taken: {(end - start) * 1e6:.3f} microseconds")

2
Time taken: 1092.916 microseconds


# Below is an optimized version of the code

In [4]:
import string

def optimized_z_array(s):
    """
    Compute the Z-array for string s using the Z algorithm (Gusfield Theorem 1.4.1).

    The Z-array is defined such that Z[k] is the length of the longest substring
    starting at k that matches a prefix of s.
    """
    n = len(s)
    assert n > 1, "Input string must have length > 1"
    z = [n] + [0] * (n - 1)
    
    # Compute initial z[1] by comparing s[1:] with prefix
    for i in range(1, n):
        if s[i] == s[i - 1]:
            z[1] += 1
        else:
            break

    l, r = (1, z[1]) if z[1] > 0 else (0, 0)
    
    for k in range(2, n):
        if k > r:
            # Case 1: No previous info available; match from scratch.
            while k + z[k] < n and s[z[k]] == s[k + z[k]]:
                z[k] += 1
            if z[k] > 0:
                l, r = k, k + z[k] - 1
        else:
            # Case 2: Use previously computed values.
            beta_length = r - k + 1
            z_k_l = z[k - l]
            if beta_length > z_k_l:
                z[k] = z_k_l
            else:
                # Extend the match beyond r.
                start = r + 1
                while start < n and s[start] == s[start - k]:
                    start += 1
                z[k] = start - k
                l, r = k, start - 1
    return z


def optimized_n_array(s):
    """
    Compute the N-array for string s.
    
    The N-array is constructed by reversing s, computing its Z-array, and then reversing the result.
    """
    return optimized_z_array(s[::-1])[::-1]


def optimized_big_l_prime_array(p, n):
    """
    Compute the L' array for pattern p using its N-array n.
    
    L'[i] = largest index j less than len(p) such that N[j] == |p[i:]|
    """
    m = len(p)
    lp = [0] * m
    for j in range(m - 1):
        i = m - n[j]
        if i < m:
            lp[i] = j + 1
    return lp


def optimized_big_l_array(p, lp):
    """
    Compute the L array for pattern p using the L' array.
    
    L[i] = largest index j less than len(p) such that N[j] >= |p[i:]|
    """
    m = len(p)
    l_arr = [0] * m
    l_arr[1] = lp[1]
    for i in range(2, m):
        l_arr[i] = max(l_arr[i - 1], lp[i])
    return l_arr


def optimized_small_l_prime_array(n):
    """
    Compute the small l' array from the N-array n.
    
    small_l_prime[i] gives the length of the longest substring starting at i that is also a prefix.
    """
    m = len(n)
    small_lp = [0] * m
    for i in range(m):
        if n[i] == i + 1:
            small_lp[m - i - 1] = i + 1
    for i in range(m - 2, -1, -1):
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i + 1]
    return small_lp


def optimized_good_suffix_table(p):
    """
    Compute the tables needed for the good suffix rule for pattern p.
    
    Returns:
        - Big L' array,
        - Big L array,
        - Small l' array.
    """
    n = optimized_n_array(p)
    lp = optimized_big_l_prime_array(p, n)
    return lp, optimized_big_l_array(p, lp), optimized_small_l_prime_array(n)


def optimized_good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """
    Given a mismatch at offset i in the pattern, determine the shift amount based on the good suffix rule.
    
    Args:
        i: The index of mismatch in the pattern.
        big_l_prime: The Big L' array.
        small_l_prime: The small l' array.
    
    Returns:
        The number of positions to shift the pattern.
    """
    m = len(big_l_prime)
    if i == m - 1:
        return 0
    i += 1  # Adjust to point to the leftmost matching position of the pattern.
    if big_l_prime[i] > 0:
        return m - big_l_prime[i]
    return m - small_l_prime[i]


def optimized_good_suffix_match(small_l_prime):
    """
    Given a full match of the pattern in the text, determine the shift amount based on the good suffix rule.
    
    Args:
        small_l_prime: The small l' array.
    
    Returns:
        The number of positions to shift the pattern.
    """
    return len(small_l_prime) - small_l_prime[1]


def optimized_dense_bad_char_tab(p, amap):
    """
    Build a dense bad character table for pattern p given an alphabet mapping (amap).
    
    The table is a list of lists, indexed by pattern position then by character index.
    """
    tab = []
    nxt = [0] * len(amap)
    for i, c in enumerate(p):
        tab.append(nxt.copy())
        nxt[amap[c]] = i + 1
    return tab


class OptimizedBoyerMoore:
    """
    An optimized implementation of the Boyer-Moore string matching algorithm.
    
    This class precomputes the bad character table and good suffix tables for a given pattern.
    """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        self.amap = {c: i for i, c in enumerate(alphabet)}
        self.bad_char = optimized_dense_bad_char_tab(p, self.amap)
        _, self.big_l, self.small_l_prime = optimized_good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """
        Compute the shift based on the bad character rule given a mismatch at offset i with character c.
        
        Args:
            i: The index of the mismatch in the pattern.
            c: The mismatching character in the text.
        
        Returns:
            The number of positions to shift the pattern.
        """
        ci = self.amap[c]
        return i - (self.bad_char[i][ci] - 1)
    
    def good_suffix_rule(self, i):
        """
        Compute the shift based on the good suffix rule for a mismatch at offset i.
        
        Args:
            i: The index of the mismatch in the pattern.
        
        Returns:
            The number of positions to shift the pattern.
        """
        m = len(self.p)
        if i == m - 1:
            return 0
        i += 1  # Adjust to point to leftmost matching position.
        if self.big_l[i] > 0:
            return m - self.big_l[i]
        return m - self.small_l_prime[i]
    
    def match_skip(self):
        """
        Compute the shift to use when a full match of the pattern is found in the text.
        
        Returns:
            The shift amount based on the good suffix rule.
        """
        return len(self.small_l_prime) - self.small_l_prime[1]


In [8]:
# GCTAGCTCTACGAGTCTA
import time
start = time.perf_counter()
p = 'TCAA'
p_bm = OptimizedBoyerMoore(p, alphabet='ACGT')
print(p_bm.bad_character_rule(2, 'T'))
end = time.perf_counter()
print(f"Time taken: {(end - start) * 1e6:.3f} microseconds")

2
Time taken: 569.792 microseconds


### Good Suffix Rule

In [13]:
import time
start = time.perf_counter()
# GCTAGCTCTACGAGTCTA
p = 'ACTA'
p_bm = BoyerMoore(p, alphabet='ACGT')
print(p_bm.good_suffix_rule(0))

end = time.perf_counter()
print(f"Time taken: {(end - start) * 1e6:.3f} microseconds")


import time
start = time.perf_counter()
# GCTAGCTCTACGAGTCTA
p = 'ACTA'
p_bm = OptimizedBoyerMoore(p, alphabet='ACGT')
print(p_bm.good_suffix_rule(0))
end = time.perf_counter()
print(f"Time taken with optimization: {(end - start) * 1e6:.3f} microseconds")

3
Time taken: 457.833 microseconds
3
Time taken with optimization: 165.416 microseconds


### Match Skip

In [14]:
import time
start = time.perf_counter()
# GCTAGCTCTACGAGTCTA
p = 'ACAC'
p_bm = BoyerMoore(p, alphabet='ACGT')
print(p_bm.match_skip())

end = time.perf_counter()
print(f"Time taken: {(end - start) * 1e6:.3f} microseconds")


import time
start = time.perf_counter()
# GCTAGCTCTACGAGTCTA
p = 'ACAC'
p_bm = OptimizedBoyerMoore(p, alphabet='ACGT')
print(p_bm.match_skip())
end = time.perf_counter()
print(f"Time taken with optimization: {(end - start) * 1e6:.3f} microseconds")

2
Time taken: 754.625 microseconds
2
Time taken with optimization: 178.459 microseconds


# Implement Boyer Moore

In [15]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

def optimized_boyer_moore(p, p_bm, t):
    """Do Boyer-Moore matching (optimized version).

    Args:
        p (str): The pattern string.
        p_bm (BoyerMoore): A preprocessed Boyer-Moore object for the pattern.
        t (str): The text in which to search for the pattern.

    Returns:
        list: A list of starting indices where the pattern is found in the text.
    """
    m = len(p)
    n = len(t)
    i = 0
    occurrences = []
    
    while i <= n - m:
        shift = 1
        mismatched = False
        # Compare pattern p and text t from rightmost end of p
        for j in range(m - 1, -1, -1):
            if p[j] != t[i + j]:
                skip_bc = p_bm.bad_character_rule(j, t[i + j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            # On a full match, determine shift based on the match skip rule.
            shift = max(shift, p_bm.match_skip())
        i += shift
    return occurrences

## Now check

In [16]:
import time
start = time.perf_counter()
t = 'GCTAGCTCTACGAGTCTA'
p = 'TCTA'
p_bm = BoyerMoore(p, alphabet='ACGT')
print(boyer_moore(p, p_bm, t))

end = time.perf_counter()
print(f"Time taken: {(end - start) * 1e6:.3f} microseconds")


start = time.perf_counter()
t = 'GCTAGCTCTACGAGTCTA'
p = 'TCTA'
p_bm = OptimizedBoyerMoore(p, alphabet='ACGT')
print(optimized_boyer_moore(p, p_bm, t))
end = time.perf_counter()
print(f"Time taken with optimization: {(end - start) * 1e6:.3f} microseconds")

[6, 14]
Time taken: 1149.459 microseconds
[6, 14]
Time taken with optimization: 909.083 microseconds


# Raita Algorithm

In [17]:
def optimized_raita(p, t):
    """
    Perform pattern matching using the Raita algorithm.

    The Raita algorithm is a simplified variant of Boyer-Moore that performs quick 
    preliminary checks on key positions of the pattern—specifically the last, first, 
    and middle characters—before doing a full character-by-character comparison.
    
    Args:
        p (str): The pattern to search for.
        t (str): The text in which to search for the pattern.
        
    Returns:
        list: A list of starting indices where the pattern is found in the text.
    """
    m = len(p)
    n = len(t)
    if m > n:
        return []
    
    # Precompute key characters from the pattern
    first_char = p[0]
    mid_char = p[m // 2]
    last_char = p[m - 1]
    
    occurrences = []
    # Loop through each possible alignment of p in t
    for i in range(n - m + 1):
        # Quick check: compare last, first, and middle characters
        if t[i + m - 1] == last_char and t[i] == first_char and t[i + m // 2] == mid_char:
            # Full comparison only if quick check passes
            if t[i:i + m] == p:
                occurrences.append(i)
    return occurrences


In [19]:
start = time.perf_counter()
t = 'GCTAGCTCTACGAGTCTA'
p = 'TCTA'
p_bm = optimized_raita(p, t)
print(p_bm)
end = time.perf_counter()
print(f"Time taken with optimization: {(end - start) * 1e6:.3f} microseconds")

[6, 14]
Time taken with optimization: 436.667 microseconds


# Apostolico–Giancarlo Algorithm

In [20]:
def optimized_apostolico_giancarlo(p, t):
    """
    Perform pattern matching using the Apostolico–Giancarlo algorithm.

    This algorithm is a variant of Boyer–Moore that uses a skip array to record the number
    of characters already verified as matching from a previous alignment. With the help of
    a precomputed "last occurrence" table for the bad-character rule, the algorithm avoids
    redundant comparisons, making it efficient for matching patterns in large texts.

    Args:
        p (str): The pattern to search for.
        t (str): The text in which to search for the pattern.

    Returns:
        list: A list of starting indices where the pattern is found in the text.
    """
    m = len(p)
    n = len(t)
    occurrences = []
    # skip array holds, for each position r in the pattern (1-indexed for convenience),
    # the number of characters known to match in the previous alignment.
    skip = [0] * (m + 1)

    # Precompute the last occurrence for each character in p.
    # For any character c, last[c] is the rightmost index where c occurs in p.
    last = {}
    for index, char in enumerate(p):
        last[char] = index

    i = 0  # index in text where the current alignment of p begins
    while i <= n - m:
        j = m - 1  # start comparing from the right end of the pattern
        # Use skip array: only compare positions j down to skip[j+1]
        while j >= skip[j + 1] and p[j] == t[i + j]:
            j -= 1
        if j < skip[j + 1]:
            # Full match found; record occurrence.
            occurrences.append(i)
            # Determine shift when a match occurs.
            shift = m - skip[1]
            if shift < 1:
                shift = 1
        else:
            # Mismatch at position j.
            # Use bad character rule with precomputed last occurrence.
            # If t[i+j] is not found in p, last.get(...) returns -1.
            shift = j - last.get(t[i + j], -1)
            if shift < 1:
                shift = 1

        # Update the skip array for the next alignment.
        # For positions that will still be in the pattern after shifting, reduce their skip values.
        for k in range(1, m - shift + 1):
            skip[k] = max(0, skip[k + shift] - shift)
        # For positions that move out of the pattern, reset to 0.
        for k in range(m - shift + 1, m + 1):
            skip[k] = 0

        i += shift

    return occurrences




In [21]:
start = time.perf_counter()
# Example usage:
if __name__ == "__main__":
    t = 'GCTAGCTCTACGAGTCTA'
    p = 'TCTA'
    matches = optimized_apostolico_giancarlo(p, t)
    print("Pattern found at positions:", matches)
end = time.perf_counter()
print(f"Time taken: {(end - start) * 1e6:.3f} microseconds")

Pattern found at positions: [6, 14]
Time taken: 390.917 microseconds


# Knuth–Morris–Pratt Algorithm

In [22]:
def compute_prefix_function(p):
    """
    Compute the prefix function for pattern p.

    The prefix function for a pattern p is an array π where π[q] is the length of the
    longest prefix of p that is also a suffix of p[0:q+1]. This function is used to determine
    how far to shift the pattern when a mismatch occurs during the KMP search.

    Args:
        p (str): The pattern string for which to compute the prefix function.

    Returns:
        list: The prefix function as a list of integers.
    """
    m = len(p)
    pi = [0] * m
    k = 0  # length of the longest prefix-suffix
    # Loop through the pattern starting at the second character
    for q in range(1, m):
        while k > 0 and p[k] != p[q]:
            k = pi[k - 1]
        if p[k] == p[q]:
            k += 1
        pi[q] = k
    return pi


def kmp_search(p, t):
    """
    Perform pattern matching using the Knuth–Morris–Pratt (KMP) algorithm.

    This function searches for occurrences of the pattern p in the text t by precomputing
    a prefix function that determines how far the pattern can be shifted when a mismatch occurs.
    It returns a list of starting indices where the pattern is found in the text.

    Args:
        p (str): The pattern to search for.
        t (str): The text in which to search for the pattern.

    Returns:
        list: A list of starting indices where the pattern p occurs in the text t.
    """
    m = len(p)
    n = len(t)
    pi = compute_prefix_function(p)
    occurrences = []
    q = 0  # number of characters matched

    # Loop through each character in the text
    for i in range(n):
        while q > 0 and p[q] != t[i]:
            q = pi[q - 1]
        if p[q] == t[i]:
            q += 1
        if q == m:
            occurrences.append(i - m + 1)
            q = pi[q - 1]  # prepare for the next possible match
    return occurrences


In [23]:
start = time.perf_counter()
# Example usage:
if __name__ == "__main__":
    t = 'GCTAGCTCTACGAGTCTA'
    p = 'TCTA'
    matches = kmp_search(p, t)
    print("Pattern found at positions:", matches)
end = time.perf_counter()
print(f"Time taken: {(end - start) * 1e6:.3f} microseconds")


Pattern found at positions: [6, 14]
Time taken: 733.833 microseconds
